In [1]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import re
import time
import datetime
from bs4 import BeautifulSoup
import urllib.request
import os

# 웹 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")


In [2]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import urllib.request
import os

# JSON 파일 로드
with open('numbered_cafes.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 카페 ID 리스트 추출
cafe_ids = [cafe['id값'] for cafe in data]

# 웹 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=options)

for cafe_id in cafe_ids:
    try:
        # URL 구성
        url = f'https://map.naver.com/p/entry/place/{cafe_id}?c=15.00,0,0,0,dh&placePath=/photo'
        driver.get(url)
        driver.implicitly_wait(30)

        # iframe 전환
        driver.switch_to.frame('entryIframe')
        time.sleep(5)  # iframe이 로드되기를 기다림

        # 이미지 리스트 컨테이너 찾기
        try:
            image_list_container = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div.place_section.no_margin > div > div > div > div:nth-child(1)'))
            )
        except Exception as e:
            print(f"이미지 리스트 컨테이너를 찾지 못했습니다 (카페 ID: {cafe_id}): {e}")
            continue

        # 페이지 다운을 통한 추가 이미지 로드 (필요시)
        body = driver.find_element(By.TAG_NAME, 'body')
        for _ in range(5):  # 최대 5번 페이지 다운
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)

        # 페이지 소스 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 모든 img 태그를 찾아서 src 속성이 있는 것들만 선택
        img_tags = soup.select('#app-root div div div div:nth-child(6) div.place_section.no_margin div div div div:nth-child(1) img')

        # 이미지 저장 경로 설정
        save_dir = os.path.join("downloaded_images", f"cafe_{cafe_id}")
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # 이미지 다운로드 (모든 이미지 다운로드)
        for i, img_tag in enumerate(img_tags):
            image_url = img_tag.get('src')
            if image_url:  # src 속성이 존재할 경우에만 다운로드
                file_name = os.path.join(save_dir, f'image_{i}.jpg')
                urllib.request.urlretrieve(image_url, file_name)
                print(f"Downloaded {file_name} for cafe ID {cafe_id}")

    except Exception as e:
        print(f"오류 발생 (카페 ID: {cafe_id}): {e}")

    finally:
        driver.switch_to.default_content()  # 다음 카페 ID로 넘어가기 전에 메인 콘텐츠로 돌아가기

# 브라우저 종료
driver.quit()

print("모든 카페의 이미지 크롤링 및 저장 완료")


이미지 리스트 컨테이너를 찾지 못했습니다 (카페 ID: 1127892000): Message: 
Stacktrace:
0   chromedriver                        0x00000001008b5088 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x00000001008ad764 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x00000001004bc82c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000100500834 cxxbridge1$string$len + 367060
4   chromedriver                        0x000000010053848c cxxbridge1$string$len + 595500
5   chromedriver                        0x00000001004f5474 cxxbridge1$string$len + 321044
6   chromedriver                        0x00000001004f60e4 cxxbridge1$string$len + 324228
7   chromedriver                        0x000000010087ca6c cxxbridge1$str$ptr + 1656336
8   chromedriver                        0x00000001008814c8 cxxbridge1$str$ptr + 1675372
9   chromedriver                        0x0000000100862950 cxxbridge1$str$ptr + 1549556
10  chromedriver                        0x000

In [2]:
driver = webdriver.Chrome(options=options)

# URL 구성
url = 'https://map.naver.com/p/entry/place/1536816099?c=15.00,0,0,0,dh&placePath=/photo'
driver.get(url)
driver.implicitly_wait(30)

try:
    # iframe 전환
    driver.switch_to.frame('entryIframe')
    time.sleep(5)  # iframe이 로드되기를 기다림

    # 이미지 리스트 컨테이너 찾기
    image_list_container = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div.place_section.no_margin > div > div > div > div:nth-child(1)'))
    )

    # 페이지 다운을 통한 추가 이미지 로드 (필요시)
    body = driver.find_element(By.TAG_NAME, 'body')
    for _ in range(5):  # 최대 5번 페이지 다운
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 모든 img 태그를 찾아서 src 속성이 있는 것들만 선택
    img_tags = soup.select('#app-root div div div div:nth-child(6) div.place_section.no_margin div div div div:nth-child(1) img')

    # 이미지 저장 경로 설정
    save_dir = "downloaded_images"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 이미지 다운로드 (모든 이미지 다운로드)
    for i, img_tag in enumerate(img_tags):
        image_url = img_tag.get('src')
        if image_url:  # src 속성이 존재할 경우에만 다운로드
            file_name = os.path.join(save_dir, f'image_{i}.jpg')
            try:
                urllib.request.urlretrieve(image_url, file_name)
                print(f"Downloaded {file_name}")
            except Exception as e:
                print(f"Failed to download {image_url}: {e}")

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 브라우저 종료
    driver.quit()

print("이미지 크롤링 및 저장 완료")

오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.101)
Stacktrace:
0   chromedriver                        0x0000000105205088 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x00000001051fd764 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x0000000104e0c82c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000104de7e50 core::str::slice_error_fail::he7b2aa4898bc357e + 3908
4   chromedriver                        0x0000000104e75574 cxxbridge1$string$len + 517908
5   chromedriver                        0x0000000104e87dd8 cxxbridge1$string$len + 593784
6   chromedriver                        0x0000000104e45474 cxxbridge1$string$len + 321044
7   chromedriver                        0x0000000104e460e4 cxxbridge1$string$len + 324228
8   chromedriver                        0x00000001051cca6c cxxbridge1$str$ptr + 1656336
9   chromedriver      

In [ ]:
# 검색을 통한 이미지 추출.

https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=모센즈스위트+성수&oquery=베러초이스+성수&tqi=ir2b4sqo1Lwssfn2eLZssssss6K-223192

In [4]:
import time
import os

import urllib.request
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

IMAGE = input("검색할 이미지를 입력하세요: ")
CNT = int(input("저장할 이미지 개수를 입력하세요: "))
### Chrome web 사용.
options = webdriver.ChromeOptions()                     # Chrome 창의 옵션을 설정할 수 있는 코드.
options.add_argument('--no-sandbox')                    
options.add_argument('--disable-dev-shm-usage')         
options.add_argument('--headless')                      # headless : Broswer 창을 띄우지 않고 수행 (colab에서는 필수).
# options.add_argument("window-size = 1920,1080")       # window size 설정


browser = webdriver.Chrome(options = options)           # 위 Option을 적용한 Chrome 창을 실행.
browser.maximize_window()                               # browser 창 최대화

### 이미지를 검색할 페이지 이동
url = "https://www.naver.com"
browser.get(url)

### 이미지 검색
elem = browser.find_element(By.NAME, "query")
elem.send_keys(IMAGE)
elem.send_keys(Keys.ENTER)
time.sleep(3)   # 검색 완료까지 대기. 

### 이미지 탭 클릭
elem = browser.find_element(By.XPATH, '//*[@id="lnb"]/div[1]/div/div[1]/div/div[1]/div[3]/a' )
elem.click()
time.sleep(3)




In [7]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# 네이버 모바일 검색 URL 패턴
search_url = "https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query={}"

# 카페 목록 (JSON 파일로부터 읽은 데이터)
cafes = [
    {"번호": 1, "카페명": "오푸 성수"},
    {"번호": 2, "카페명": "모센즈스위트 성수"},
    # 나머지 카페들...
]

def crawl_images(cafe_name):
    # 카페명을 URL에 맞게 인코딩
    encoded_cafe_name = urllib.parse.quote(cafe_name)
    url = search_url.format(encoded_cafe_name)

    # 네이버 검색 결과 페이지 요청
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve search results for {cafe_name}")
        return []

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')

    # 첫 번째 selector로 이미지 태그를 선택하여 src 속성 추출
    image_tags_1 = soup.select('img.K0PDV._div')
    image_urls_1 = [img['src'] for img in image_tags_1 if 'src' in img.attrs]

    # 첫 번째 리스트의 마지막 이미지를 제외
    if image_urls_1:
        image_urls_1 = image_urls_1[:-1]

    # 두 번째 selector로 음식 사진 이미지 태그를 선택하여 src 속성 추출
    image_tags_2 = soup.select('#place-main-section-root > div > div:nth-child(4) > div.place_section_content > ul > li > a > div.ZHqBk > div > div.lazyload-wrapper img.K0PDV')
    image_urls_2 = [img['src'] for img in image_tags_2 if 'src' in img.attrs]

    # 두 리스트를 합쳐서 반환
    all_image_urls = image_urls_1 + image_urls_2
    return all_image_urls

# 각 카페별로 이미지 크롤링 수행
for cafe in cafes:
    cafe_name = cafe["카페명"]
    print(f"Crawling images for: {cafe_name}")
    image_urls = crawl_images(cafe_name)
    for idx, url in enumerate(image_urls):
        print(f"Image {idx + 1}: {url}")


Crawling images for: 오푸 성수
Image 1: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230418_206%2F1681744751969cj4AO_PNG%2F%25BD%25BA%25C5%25A9%25B8%25B0%25BC%25A6_2023-04-18_%25BF%25C0%25C0%25FC_12.17.59.png
Image 2: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20231208_266%2F1702014763303lqL8W_JPEG%2F%25C5%25AB%25C4%25A7%25B4%25EB%25B9%25DD.jpg
Crawling images for: 모센즈스위트 성수
Image 1: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221117_90%2F1668645854289hCaW0_JPEG%2FKakaoTalk_Photo_2022-11-17-09-43-51_004.jpeg
Image 2: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230614_131%2F1686730256947WWo2u_JPEG%2FIMG_1474.JPG


In [10]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# 네이버 모바일 검색 URL 패턴
search_url = "https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query={}"

# 카페 목록 (JSON 파일로부터 읽은 데이터)
cafes = [
    {"번호": 1, "카페명": "오푸 성수"},
    {"번호": 2, "카페명": "모센즈스위트 성수"},
    # 나머지 카페들...
]

def crawl_images(cafe_name):
    # 카페명을 URL에 맞게 인코딩
    encoded_cafe_name = urllib.parse.quote(cafe_name)
    url = search_url.format(encoded_cafe_name)

    # 네이버 검색 결과 페이지 요청
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve search results for {cafe_name}")
        return []

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')

    # 첫 번째 selector로 이미지 태그를 선택하여 src 속성 추출
    image_tags_1 = soup.select('img.K0PDV._div')
    image_urls_1 = [img['src'] for img in image_tags_1 if 'src' in img.attrs]

    # 첫 번째 리스트의 마지막 이미지를 제외
    if image_urls_1:
        image_urls_1 = image_urls_1[:-1]

    # 두 번째 selector로 음식 사진 이미지 태그를 선택하여 src 속성 추출
    # 주어진 XPath에 해당하는 부분을 탐색
    ul_element = soup.select_one('#place-main-section-root > div > div:nth-child(4) > div.place_section_content > ul')
    if ul_element:
        image_tags_2 = ul_element.select('div.lazyload-wrapper img')
        image_urls_2 = [img['src'] for img in image_tags_2 if 'src' in img.attrs]
    else:
        image_urls_2 = []

    # 두 리스트를 합쳐서 반환
    all_image_urls = image_urls_1 + image_urls_2
    return all_image_urls

# 각 카페별로 이미지 크롤링 수행
for cafe in cafes:
    cafe_name = cafe["카페명"]
    print(f"Crawling images for: {cafe_name}")
    image_urls = crawl_images(cafe_name)
    for idx, url in enumerate(image_urls):
        print(f"Image {idx + 1}: {url}")


Crawling images for: 오푸 성수
Image 1: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230418_206%2F1681744751969cj4AO_PNG%2F%25BD%25BA%25C5%25A9%25B8%25B0%25BC%25A6_2023-04-18_%25BF%25C0%25C0%25FC_12.17.59.png
Image 2: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20231208_266%2F1702014763303lqL8W_JPEG%2F%25C5%25AB%25C4%25A7%25B4%25EB%25B9%25DD.jpg
Crawling images for: 모센즈스위트 성수
Image 1: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221117_90%2F1668645854289hCaW0_JPEG%2FKakaoTalk_Photo_2022-11-17-09-43-51_004.jpeg
Image 2: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230614_131%2F1686730256947WWo2u_JPEG%2FIMG_1474.JPG


In [26]:
import requests
from lxml import html
import json
import os

# JSON 파일 경로
json_file_path = '/Users/macbook/Desktop/numbered_cafes.json'

# 이미지 저장 경로
image_save_path = '/Users/macbook/Desktop/cafe_images'

# 디렉토리가 없으면 생성
if not os.path.exists(image_save_path):
    os.makedirs(image_save_path)

# JSON 파일에서 카페 목록 로드
with open(json_file_path, 'r', encoding='utf-8') as f:
    cafes = json.load(f)

# 네이버 모바일 검색 URL 패턴
search_url = "https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query={}"

def crawl_images(cafe_name):
    # 카페명을 URL에 맞게 인코딩
    encoded_cafe_name = requests.utils.quote(cafe_name)
    url = search_url.format(encoded_cafe_name)

    # 네이버 검색 결과 페이지 요청
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve search results for {cafe_name}")
        return []

    # lxml의 html parser를 이용하여 HTML 파싱
    tree = html.fromstring(response.content)

    # XPath로 모든 <div class="place_thumb"> 요소 찾기
    img_xpath = '//*[@id="place-main-section-root"]/div/div[4]/div[1]/ul//div[@class="place_thumb"]/img'
    image_tags = tree.xpath(img_xpath)

    # img 태그의 src 속성 추출
    image_urls = [img.get('src') for img in image_tags if img.get('src')]

    # 반환된 이미지 URL 리스트 출력
    return image_urls

# 각 카페별로 이미지 크롤링 수행
for cafe in cafes:
    cafe_name = cafe["카페명"]
    print(f"Crawling images for: {cafe_name}")
    image_urls = crawl_images(cafe_name)
    
    for idx, url in enumerate(image_urls):
        print(f"Image {idx + 1}: {url}")
        # 이미지 다운로드 및 저장
        img_data = requests.get(url).content
        image_filename = os.path.join(image_save_path, f"{cafe_name}_{idx + 1}.jpg")
        with open(image_filename, 'wb') as handler:
            handler.write(img_data)
        print(f"Saved image {idx + 1} for {cafe_name} to {image_filename}")


Crawling images for: 오푸 성수
Failed to retrieve search results for 오푸 성수
Crawling images for: 모센즈스위트 성수
Failed to retrieve search results for 모센즈스위트 성수
Crawling images for: 어피스앤드피스 성수
Failed to retrieve search results for 어피스앤드피스 성수
Crawling images for: 베러초이스 성수
Failed to retrieve search results for 베러초이스 성수
Crawling images for: 에젤커피 성수
Failed to retrieve search results for 에젤커피 성수
Crawling images for: 라에비뉴 성수
Failed to retrieve search results for 라에비뉴 성수
Crawling images for: 나드에프앤비 성수
Failed to retrieve search results for 나드에프앤비 성수
Crawling images for: 카페시벳 성수
Failed to retrieve search results for 카페시벳 성수
Crawling images for: 리얼 성수
Failed to retrieve search results for 리얼 성수
Crawling images for: 로우키 성수
Failed to retrieve search results for 로우키 성수
Crawling images for: 니드 성수
Failed to retrieve search results for 니드 성수
Crawling images for: 높은산 성수
Failed to retrieve search results for 높은산 성수
Crawling images for: 라스베이글 성수
Failed to retrieve search results for 라스베이글 성수
Crawling images for: 언니

최종 이미지 코드!

In [19]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import json

# JSON 파일 경로
json_file_path = '/Users/macbook/Desktop/numbered_cafes.json'

# JSON 파일에서 카페 목록 로드
with open(json_file_path, 'r', encoding='utf-8') as f:
    cafes = json.load(f)

# 네이버 모바일 검색 URL 패턴
search_url = "https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query={}"

def crawl_images(cafe_name):
    # 카페명을 URL에 맞게 인코딩
    encoded_cafe_name = urllib.parse.quote(cafe_name)
    url = search_url.format(encoded_cafe_name)

    # 네이버 검색 결과 페이지 요청
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve search results for {cafe_name}")
        return []

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')

    # 첫 번째 selector로 이미지 태그를 선택하여 src 속성 추출
    image_tags_1 = soup.select('img.K0PDV._div')
    image_urls_1 = [img['src'] for img in image_tags_1 if 'src' in img.attrs]

    # 첫 번째 리스트의 마지막 이미지를 제외
    if image_urls_1:
        image_urls_1 = image_urls_1[:-1]

    # 두 번째 selector로 음식 사진 이미지 태그를 선택하여 src 속성 추출
    ul_element = soup.select_one('#place-main-section-root > div > div:nth-child(4) > div.place_section_content > ul')
    if ul_element:
        image_tags_2 = ul_element.select('div.lazyload-wrapper img')
        image_urls_2 = [img['src'] for img in image_tags_2 if 'src' in img.attrs]
    else:
        image_urls_2 = []

    # 두 리스트를 합쳐서 반환
    all_image_urls = image_urls_1 + image_urls_2
    return all_image_urls

# 각 카페별로 이미지 크롤링 수행
for cafe in cafes:
    cafe_name = cafe["카페명"]
    print(f"Crawling images for: {cafe_name}")
    image_urls = crawl_images(cafe_name)
    for idx, url in enumerate(image_urls):
        print(f"Image {idx + 1}: {url}")


Crawling images for: 오푸 성수
Image 1: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230418_206%2F1681744751969cj4AO_PNG%2F%25BD%25BA%25C5%25A9%25B8%25B0%25BC%25A6_2023-04-18_%25BF%25C0%25C0%25FC_12.17.59.png
Image 2: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20231208_266%2F1702014763303lqL8W_JPEG%2F%25C5%25AB%25C4%25A7%25B4%25EB%25B9%25DD.jpg
Crawling images for: 모센즈스위트 성수
Image 1: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221117_90%2F1668645854289hCaW0_JPEG%2FKakaoTalk_Photo_2022-11-17-09-43-51_004.jpeg
Image 2: https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230614_131%2F1686730256947WWo2u_JPEG%2FIMG_1474.JPG
Crawling images for: 어피스앤드피스 성수
Image 1: https://search.pstatic.net/common/?autoRotate=true&quality

인스타그램 피드 이미지 크롤링

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# 크롤링할 인스타그램 url 설정
url = "https://www.instagram.com/mohssenssweet_seongsu/"

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 게시물 내 이미지 클래스명으로 찾기
image = driver.find_element(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장
image_src = image.get_attribute('src')

# 이미지 다운로드
response = requests.get(image_src)
filename = 'image.jpg'
with open(filename, 'wb+') as f:
    f.write(response.content)

print("사진 수집이 완료되었습니다.")


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?

인스타 피드 이미지 크롤링

#1 오푸 성수

In [3]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# 크롤링할 인스타그램 url 설정
url = "https://www.instagram.com/oafuoafu"

# 이미지를 저장할 폴더 이름 설정
folder_name = "오푸_instagram_images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    response = requests.get(image_src)
    filename = os.path.join(folder_name, f'오푸_image_{idx}.jpg')
    with open(filename, 'wb') as f:
        f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 '오푸_instagram_images' 폴더에 저장되었습니다.


#2 모센즈스위트 성수

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# 크롤링할 인스타그램 url 설정
url = "https://www.instagram.com/mohssenssweet_seongsu/"

# 이미지를 저장할 폴더 이름 설정
folder_name = "모센즈스위트_instagram_images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    response = requests.get(image_src)
    filename = os.path.join(folder_name, f'모센즈스위트_image_{idx}.jpg')
    with open(filename, 'wb') as f:
        f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 '모센즈스위트_instagram_images' 폴더에 저장되었습니다.


In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# 크롤링할 인스타그램 url 설정
url = "https://www.instagram.com/mohssenssweet_seongsu/"

# 이미지를 저장할 폴더 이름 설정
folder_name = "모센즈스위트_instagram_images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    response = requests.get(image_src)
    filename = os.path.join(folder_name, f'모센즈스위트_image_{idx}.jpg')
    with open(filename, 'wb') as f:
        f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

#3 어피스앤드피스

In [4]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# 크롤링할 인스타그램 url 설정
url = "https://www.instagram.com/apieceandpeace"

# 이미지를 저장할 폴더 이름 설정
folder_name = "어피스앤드피스_instagram_images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    response = requests.get(image_src)
    filename = os.path.join(folder_name, f'어피스앤드피스_image_{idx}.jpg')
    with open(filename, 'wb') as f:
        f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 '어피스앤드피스_instagram_images' 폴더에 저장되었습니다.


#4 베러초이스

In [5]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time

# 크롤링할 인스타그램 url 설정
url = "https://www.instagram.com/betterchoice_korea"

# 이미지를 저장할 폴더 이름 설정
folder_name = "instagram_images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 20
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src not in collected_images:
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    response = requests.get(image_src)
    filename = os.path.join(folder_name, f'image_{idx}.jpg')
    with open(filename, 'wb') as f:
        f.write(response.content)

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 20장의 사진이 'instagram_images' 폴더에 저장되었습니다.


#5 EZER Coffee

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# 크롤링할 인스타그램 url 설정
url = "https://www.instagram.com/ezercoffee/"

# 이미지를 저장할 폴더 이름 설정
folder_name = "EZER Coffee_instagram_images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    response = requests.get(image_src)
    filename = os.path.join(folder_name, f'EZER Coffee_image_{idx}.jpg')
    with open(filename, 'wb') as f:
        f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'EZER Coffee_instagram_images' 폴더에 저장되었습니다.


#6 라에비뉴

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/laavenue_official_/"

# 이미지 파일명 관련 변수 설정
num = '6'
cafe_name = '라에비뉴'

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/6_라에비뉴_instagram_images' 폴더에 저장되었습니다.


#7 나드에프앤비, #8 카페시벳-- 직접 사진 추출

#9 naver_review_9_성수동 리얼 로스팅 커피&디저트_1190505473.json

In [7]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_9_성수동 리얼 로스팅 커피&디저트_1190505473.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/seongsureal_official"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

KeyboardInterrupt: 

# 10 naver_review_10_로우키_1393890643.json

In [8]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_10_로우키_1393890643.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/lowkey_coffee"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/10_로우키_instagram_images' 폴더에 저장되었습니다.


#11 naver_review_11_니드인_1935137830.json

In [9]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_11_니드인_1935137830.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/lowkey_coffee"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/11_니드인_instagram_images' 폴더에 저장되었습니다.


#12 naver_review_12_높은산_1081045129.json

In [19]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_12_높은산_1081045129.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/noppensan"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/12_높은산_instagram_images' 폴더에 저장되었습니다.


#13 naver_review_13_라스베이글 성수점_1173495456.json

In [17]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_13_라스베이글 성수점_1173495456.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/las.bagel/"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/13_라스베이글 성수점_instagram_images' 폴더에 저장되었습니다.


#14 naver_review_14_언니커피_1346789087.json

In [20]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_14_언니커피_1346789087.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/yejisun0812/"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/14_언니커피_instagram_images' 폴더에 저장되었습니다.


naver_review_15_파티세리 후르츠_1182897501.json

In [25]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_15_파티세리 후르츠_1182897501.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/patisserie.fruits/"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/15_파티세리 후르츠_instagram_images' 폴더에 저장되었습니다.


naver_review_16_브로벨커피_1428046376.json

In [26]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_16_브로벨커피_1428046376.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/brobellcoffee"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/16_브로벨커피_instagram_images' 폴더에 저장되었습니다.


naver_review_17_ONE BEAN카페_1782682179.json -- 직접

naver_review_18_스텀필즈커피_1765995009.json -- 직접

naver_review_19_도토루커피os_37049427.json

In [27]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_19_도토루커피os_37049427.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/coffeos"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/19_도토루커피os_instagram_images' 폴더에 저장되었습니다.


naver_review_20_카페수다_34351850.json -- 직접
naver_review_21_팔롬비니_1846703943.json -- 직접


naver_review_22_카페조이_32812333.json

In [28]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_22_카페조이_32812333.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/joecool_kkj"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/22_카페조이_instagram_images' 폴더에 저장되었습니다.


naver_review_23_투달러스커피_75758743.json -- 직접
naver_review_24_카페라샤워_1590690124.json -- 직접
naver_review_25_카페아모렘_1403886445.json

In [ ]:
naver_review_26_예빈당_11861121.json

In [29]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_26_예빈당_11861121.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/yebindang_cafe"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/26_예빈당_instagram_images' 폴더에 저장되었습니다.


naver_review_27_피어커피_33704135.json

In [30]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_27_피어커피_33704135.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/peer_coffee/"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/27_피어커피_instagram_images' 폴더에 저장되었습니다.


naver_review_28_브루잉 세레모니_1613470297.json

In [31]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_28_브루잉 세레모니_1613470297.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/brewingceremony/"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/28_브루잉 세레모니_instagram_images' 폴더에 저장되었습니다.


naver_review_29_이트앤드링크 성수_1483166635.json

In [32]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_29_이트앤드링크 성수_1483166635.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/eatanddrinkseoul"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/29_이트앤드링크 성수_instagram_images' 폴더에 저장되었습니다.


naver_review_30_아이오쓰리 에이스하이엔드점_1623328599.json -- 직접


naver_review_31_봄의정원 성수점_1502355065.json

In [33]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_31_봄의정원 성수점_1502355065.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/spring__garden_"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/31_봄의정원 성수점_instagram_images' 폴더에 저장되었습니다.


naver_review_32_마이쥬스 성수낙낙점_1626027648.json

naver_review_33_예셰숄_1984160681.json

In [34]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_33_예셰숄_1984160681.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/yesyesyall_"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/33_예셰숄_instagram_images' 폴더에 저장되었습니다.


naver_review_34_쇼콜라쏭즈_1241528957.json

In [35]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_34_쇼콜라쏭즈_1241528957.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/chocolat111_"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/34_쇼콜라쏭즈_instagram_images' 폴더에 저장되었습니다.


naver_review_35_글리터_1973759403.json

In [36]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_35_글리터_1973759403.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/glitter_seongsu"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/35_글리터_instagram_images' 폴더에 저장되었습니다.


naver_review_36_삼옥_1297713401.json

In [37]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_36_삼옥_1297713401.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/sam_ok.official"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/36_삼옥_instagram_images' 폴더에 저장되었습니다.


naver_review_37_카페리빈 성수점_1879884818.json -- 직접

naver_review_38_원스카페_1879463803.json

naver_review_39_유니드카페_1577691489.json

naver_review_40_퓨엘커피_1776731967.json

naver_review_41_커피사피엔스 성수우림점_1824108106.json

naver_review_42_키쿠키앤커피 성수_1716355636.json

naver_review_43_와이덴 성수_1982767346.json

naver_review_44_구테로이테 성수센트럴키친_1046248956.json

naver_review_45_그레이덕_1167958425.json

In [38]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_45_그레이덕_1167958425.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/grayduck_41?utm_source=qr&igshid=MzNlNGNkZWQ4Mg%3D%3D"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/45_그레이덕_instagram_images' 폴더에 저장되었습니다.


naver_review_46_누와_1612076914.json --직접

naver_review_47_카페소림_1194892405.json --직접

naver_review_48_커피냅로스터스 성수_1355804064.json -- 25개

In [39]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/coffeenap_roasters/"

# JSON 파일 이름 예시
json_file_name = "naver_review_48_커피냅로스터스 성수_1355804064.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 25
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 25장의 사진이 'cafeinsta_images/48_커피냅로스터스 성수_instagram_images' 폴더에 저장되었습니다.


naver_review_49_도렐 성수점_1432336976.json

In [40]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_49_도렐 성수점_1432336976.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/dorrell_coffee/"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/49_도렐 성수점_instagram_images' 폴더에 저장되었습니다.


naver_review_50_오우드 성수_1216172292.json

In [41]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/oude_seoul"

# JSON 파일 이름 예시
json_file_name = "naver_review_50_오우드 성수_1216172292.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 25
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 25장의 사진이 'cafeinsta_images/50_오우드 성수_instagram_images' 폴더에 저장되었습니다.


naver_review_51_기와르_1273097610.json

In [42]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/_kiwaru"

# JSON 파일 이름 예시
json_file_name = "naver_review_51_기와르_1273097610.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 25
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 25장의 사진이 'cafeinsta_images/51_기와르_instagram_images' 폴더에 저장되었습니다.


naver_review_52_베티버 성수_1383256753.json

In [43]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/v.etiver"

# JSON 파일 이름 예시
json_file_name = "naver_review_52_베티버 성수_1383256753.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 25
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 25장의 사진이 'cafeinsta_images/52_베티버 성수_instagram_images' 폴더에 저장되었습니다.


naver_review_53_Pauline CoffeeBar_1215237779.json

In [44]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_53_Pauline CoffeeBar_1215237779.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/pauline_coffeebar/"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/53_Pauline CoffeeBar_instagram_images' 폴더에 저장되었습니다.


naver_review_54_누데이크 성수_1691420621.json -- 직접

naver_review_55_젤라또투유_1247801780.json

In [45]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

# JSON 파일 이름 예시
json_file_name = "naver_review_55_젤라또투유_1247801780.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 크롤링할 인스타그램 URL 설정 (URL을 지정해야 함)
url = "https://www.instagram.com/seongsu_gelato2u/"

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본 설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 클래스명으로 모든 이미지 요소 찾기
images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")

# 이미지 주소 저장 및 다운로드
for idx, image in enumerate(images):
    image_src = image.get_attribute('src')
    if image_src:  # 이미지 소스가 존재하는 경우에만 다운로드 시도
        response = requests.get(image_src)
        filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')  # idx에 1을 더해서 1부터 시작
        with open(filename, 'wb') as f:
            f.write(response.content)

print(f"사진 수집이 완료되었습니다. 모든 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 모든 사진이 'cafeinsta_images/55_젤라또투유_instagram_images' 폴더에 저장되었습니다.


naver_review_56_마를리_742676592.json

In [46]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/marly.official_"

# JSON 파일 이름 예시
json_file_name = "naver_review_56_마를리_742676592.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 25
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 25장의 사진이 'cafeinsta_images/56_마를리_instagram_images' 폴더에 저장되었습니다.


naver_review_57_어니언 성수_38561949.json

naver_review_58_쎈느Scene_1030871168.json

In [47]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/sceneseoul_official"

# JSON 파일 이름 예시
json_file_name = "naver_review_58_쎈느Scene_1030871168.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 25
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 9}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 25장의 사진이 'cafeinsta_images/58_쎈느Scene_instagram_images' 폴더에 저장되었습니다.


naver_review_59_아이오쓰리 에이스하이엔드점_1623328599.json -- 30번이랑 겹침

naver_review_60_카멜커피 성수점_55585656.json

In [49]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/camelcoffee_kor/"

# JSON 파일 이름 예시
json_file_name = "naver_review_60_카멜커피 성수점_55585656.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/60_카멜커피 성수점_instagram_images' 폴더에 저장되었습니다.


naver_review_61_스프레이하우스_1300253897.json

In [50]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/sprayhouse_cafe"

# JSON 파일 이름 예시
json_file_name = "naver_review_61_스프레이하우스_1300253897.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/61_스프레이하우스_instagram_images' 폴더에 저장되었습니다.


naver_review_62_야페_1701590225.json

In [51]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/cafe.yafe"

# JSON 파일 이름 예시
json_file_name = "naver_review_62_야페_1701590225.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/62_야페_instagram_images' 폴더에 저장되었습니다.


naver_review_63_프레이커피바 PCB서울_1852410455.json

In [52]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/pcb_seoul"

# JSON 파일 이름 예시
json_file_name = "naver_review_63_프레이커피바 PCB서울_1852410455.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/63_프레이커피바 PCB서울_instagram_images' 폴더에 저장되었습니다.


naver_review_64_러프러프_1829908685.json

In [53]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/rufruf_museum/"

# JSON 파일 이름 예시
json_file_name = "naver_review_64_러프러프_1829908685.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/64_러프러프_instagram_images' 폴더에 저장되었습니다.


naver_review_65_카페 스트랏_1410613160.json

naver_review_66_업사이드커피_1259420360.json

In [54]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/up.side_"

# JSON 파일 이름 예시
json_file_name = "naver_review_66_업사이드커피_1259420360.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/66_업사이드커피_instagram_images' 폴더에 저장되었습니다.


naver_review_67_카페바로_1916513487.json

naver_review_68_훔볼트_1504347953.json

naver_review_69_그레이박스_1949527561.json

naver_review_70_성수동대림창고갤러리_37910590.json

naver_review_71_리브릭_1424869222.json

In [55]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/libreak"

# JSON 파일 이름 예시
json_file_name = "naver_review_71_리브릭_1424869222.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/71_리브릭_instagram_images' 폴더에 저장되었습니다.


naver_review_72_굴림_1397836032.json

In [56]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/goollimm"

# JSON 파일 이름 예시
json_file_name = "naver_review_72_굴림_1397836032.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/72_굴림_instagram_images' 폴더에 저장되었습니다.


naver_review_73_하루앤원데이_38692829.json --직접

naver_review_74_플록스_1515867867.json

naver_review_75_아쿠아산타 성수카페_1517487358.json

In [59]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/acqua__santa/"

# JSON 파일 이름 예시
json_file_name = "naver_review_75_아쿠아산타 성수카페_1517487358.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/75_아쿠아산타 성수카페_instagram_images' 폴더에 저장되었습니다.


naver_review_76_앙골로에스프레소_1066610472.json

naver_review_77_창창커피_1005128677.json

In [60]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/changchang_coffee"

# JSON 파일 이름 예시
json_file_name = "naver_review_77_창창커피_1005128677.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/77_창창커피_instagram_images' 폴더에 저장되었습니다.


naver_review_78_BNHR_1223354536.json

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/bnhrcoffee"

# JSON 파일 이름 예시
json_file_name = "naver_review_78_BNHR_1223354536.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/78_BNHR_instagram_images' 폴더에 저장되었습니다.


naver_review_79_단과자_1177180342.json

In [3]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/daan_kkaka"

# JSON 파일 이름 예시
json_file_name = "naver_review_79_단과자_1177180342.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_22_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

KeyboardInterrupt: 

naver_review_80_라니커피_1227111236.json

naver_review_81_럭럭카페_1168976634.json

naver_review_82_레이더 성수_1525007973.json

naver_review_83_로우커피스탠드 성수점_1168698330.json

In [4]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/rawcoffeestand_official"

# JSON 파일 이름 예시
json_file_name = "naver_review_83_로우커피스탠드 성수점_1168698330.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/83_로우커피스탠드 성수점_instagram_images' 폴더에 저장되었습니다.


naver_review_84_스테치_1180418614.json

naver_review_86_앤드밀 성수점_1943210266.json

In [5]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/andmeal_sy"

# JSON 파일 이름 예시
json_file_name = "naver_review_86_앤드밀 성수점_1943210266.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/86_앤드밀 성수점_instagram_images' 폴더에 저장되었습니다.


naver_review_88_어커_1560084717.json

In [6]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/accurofficial"

# JSON 파일 이름 예시
json_file_name = "naver_review_88_어커_1560084717.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/88_어커_instagram_images' 폴더에 저장되었습니다.


naver_review_89_어퍼룸_1964635228.json

In [7]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/upperroom_coffee_"

# JSON 파일 이름 예시
json_file_name = "naver_review_89_어퍼룸_1964635228.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/89_어퍼룸_instagram_images' 폴더에 저장되었습니다.


naver_review_90_어페어커피_1863706992.json

In [8]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/affaircoffee_"

# JSON 파일 이름 예시
json_file_name = "naver_review_90_어페어커피_1863706992.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/90_어페어커피_instagram_images' 폴더에 저장되었습니다.


naver_review_91_에이투비 Cafe&Bar_1537971823.json

In [9]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/atob_cafeandbar"

# JSON 파일 이름 예시
json_file_name = "naver_review_91_에이투비 Cafe&Bar_1537971823.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 40
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 40장의 사진이 'cafeinsta_images/91_에이투비 Cafe&Bar_instagram_images' 폴더에 저장되었습니다.


naver_review_92_오프커피_1921874344.json

In [12]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/5ff_coffee"

# JSON 파일 이름 예시
json_file_name = "naver_review_92_오프커피_1921874344.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 50
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 50장의 사진이 'cafeinsta_images/92_오프커피_instagram_images' 폴더에 저장되었습니다.


naver_review_93_카페온유_1169957780.json

In [15]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/_cafe_onu"

# JSON 파일 이름 예시
json_file_name = "naver_review_93_카페온유_1169957780.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 12
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 12장의 사진이 'cafeinsta_images/93_카페온유_instagram_images' 폴더에 저장되었습니다.


naver_review_94_유스트레스_1999678821.json

naver_review_95_치카치카_37191703.json

In [18]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/chika.seongsu"

# JSON 파일 이름 예시
json_file_name = "naver_review_95_치카치카_37191703.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 30
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 30장의 사진이 'cafeinsta_images/95_치카치카_instagram_images' 폴더에 저장되었습니다.


naver_review_96_카페메이_1210627765.json

naver_review_97_카페크라비_1872719083.json

naver_review_98_호두이즈월넛 HODOOISWALNUT_1362812488.json

naver_review_99_커피베이 성동성수점_1573389383.json

naver_review_100_커피식구성수센터_1447656721.json

In [19]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/coffee.seekoo_seongsu"

# JSON 파일 이름 예시
json_file_name = "naver_review_100_커피식구성수센터_1447656721.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 30
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 30장의 사진이 'cafeinsta_images/100_커피식구성수센터_instagram_images' 폴더에 저장되었습니다.


naver_review_101_커피코코_1853990672.json

naver_review_102_크레송_37392409.json

In [20]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import base64

# 크롤링할 인스타그램 URL 설정
url = "https://www.instagram.com/cressong_"

# JSON 파일 이름 예시
json_file_name = "naver_review_102_크레송_37392409.json"

# JSON 파일 이름에서 num과 cafe_name 추출
file_parts = json_file_name.split('_')
num = file_parts[2]
cafe_name = file_parts[3]

# 이미지 저장 폴더 설정 (cafeinsta_images 폴더 내에 저장)
base_folder = "cafeinsta_images"
folder_name = os.path.join(base_folder, f"{num}_{cafe_name}_instagram_images")

# 폴더가 존재하지 않을 경우 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# 크롬 웹드라이버 구동 및 기본설정
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.options import Options

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, necessary for Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)


# 이미지 수집 수와 스크롤 횟수 설정
target_image_count = 30
scroll_pause_time = 2  # 스크롤 후 대기 시간 (초)
collected_images = set()

while len(collected_images) < target_image_count:
    # 현재 로드된 모든 이미지 요소 찾기
    images = driver.find_elements(By.CLASS_NAME, "x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3")
    # 새롭게 발견된 이미지 주소 저장
    for image in images:
        image_src = image.get_attribute('src')
        if image_src and image_src not in collected_images:  # 이미지 소스가 None이 아닌지 확인
            collected_images.add(image_src)
            if len(collected_images) >= target_image_count:
                break
    
    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

# 이미지 다운로드
for idx, image_src in enumerate(collected_images):
    try:
        if image_src.startswith('data:image'):  # Base64 인코딩된 이미지 처리
            header, encoded = image_src.split(',', 1)
            data = base64.b64decode(encoded)
            file_extension = header.split('/')[1].split(';')[0]  # 파일 확장자 추출 (png, jpeg 등)
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.{file_extension}')
            with open(filename, 'wb') as f:
                f.write(data)
        else:  # 일반 URL 이미지 다운로드
            response = requests.get(image_src)
            response.raise_for_status()  # 요청에 실패한 경우 예외 발생
            filename = os.path.join(folder_name, f'{cafe_name}_image_{idx + 1}.jpg')
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {e}")

print(f"사진 수집이 완료되었습니다. 총 {len(collected_images)}장의 사진이 '{folder_name}' 폴더에 저장되었습니다.")

# 브라우저 종료
driver.quit()

사진 수집이 완료되었습니다. 총 30장의 사진이 'cafeinsta_images/102_크레송_instagram_images' 폴더에 저장되었습니다.


naver_review_103_턴온_1348444139.json